In [ ]:
!pip install requests
import requests
import json
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

def float_to_string(f):
  f = np.round(f,4)
  s = str(f)[2:]
  while len(s) < 4:
    s += "0"
  return s

Mounted at /content/drive


In [ ]:
#https://statsapi.web.nhl.com/api/v1/game/2018020001/linescore
#Use above for start time and OT or not OT
#convert zulu time to eastern, then get the date, otherwise a late start will end up in the next day

end_reached = False
year='2019'
game_number = .0001
list_2018 = []

while not end_reached:

  try:
    game_string = float_to_string(game_number)
    request_string = f"https://statsapi.web.nhl.com/api/v1/game/{year}02{game_string}/boxscore"
    response = requests.get(request_string)
  except:
    #Reached the end of the season
    end_reached = True
    break

  game = json.loads(response.content.decode('utf-8'))
  teams=['away', 'home']

  for team in teams:
    try:
      players = list(game['teams'][team]['players'].keys())
    except:
      end_reached = True
      print(game_number, game_string)
      break
    stats = []
    for player in players:
      try:
        game['teams'][team]['players'][player]['stats']['skaterStats']['player_id'] = player
        stats.append(game['teams'][team]['players'][player]['stats']['skaterStats'])
      except:
        try:
          #Player is a goalie or is scratched
          game['teams'][team]['players'][player]['stats']['goalieStats']['player_id'] = player
          stats.append(game['teams'][team]['players'][player]['stats']['goalieStats'])
        except:
          #Player scratched
          continue
    df = pd.DataFrame.from_dict(stats)
    df = df.set_index('player_id')
    df[f'{team}_team_name'] = game['teams'][team]['team']['name']
    df[f'{team}_team_id'] = game['teams'][team]['team']['id']
    df['season'] = year
    df['game_number'] = game_number
    list_2018.append(df)
  game_number += .0001

In [ ]:
df_2018 = pd.concat(list_2018)

In [ ]:
df_2018.sample(10)

In [ ]:
len(df_2018)

In [ ]:
from copy import Error
def process_box_scores(df):

  df['game_number'] = df['game_number'] * 10000
  df['game_number'] = df['game_number'].astype(int)

  home_teams = df_2018[df_2018['away_team_name'].isna()]
  away_teams = df_2018[df_2018['home_team_name'].isna()]
  #by_game_away = away_teams.groupby(['game_number']).sum()
  #by_game_home = home_teams.groupby(['game_number']).sum()

  num_games = by_game_away.index.argmax()

  if num_games != by_game_home.index.argmax():
    print("Error: Number of home and away games don't match")
    return False

  season = []

  for i in range(1,num_games+1,1):
    box_score = {}
    this_game_all = df[df['game_number'] == i]
    this_game_home = this_game_all[this_game_all['away_team_name'].isna()]
    this_game_away = this_game_all[this_game_all['home_team_name'].isna()]
    box_score['game_number'] = i
    try:
      box_score['home_team_name'] = this_game_home.home_team_name.iloc[0]
      box_score['away_team_name'] = this_game_away.away_team_name.iloc[0]
    except:
      print(f"Error getting the team names in game {i}")
    #print(by_game_home)
    try:
      box_score['home_goals'] = this_game_home.goals.sum()
      box_score['away_goals'] = this_game_away.goals.sum()
      box_score['home_shots'] = this_game_home[this_game_home.evenSaves.isna()].shots.sum()
      box_score['away_shots'] = this_game_away[this_game_away.evenSaves.isna()].shots.sum()
      box_score['home_hits'] = this_game_home.hits.sum()
      box_score['away_hits'] = this_game_away.hits.sum()
      box_score['home_ppg'] = this_game_home.powerPlayGoals.sum()
      box_score['away_ppg'] = this_game_away.powerPlayGoals.sum()
      box_score['home_goals'] = this_game_home.goals.sum()
      box_score['away_goals'] = this_game_away.goals.sum()
    except:
      print("Error getting goals")
      print()
      return False
    season.append(box_score)

  return pd.DataFrame(season)
  


In [ ]:
df_2018 = pd.read_csv('/content/drive/MyDrive/2018_box_scores.csv')
new_df = process_box_scores(df_2018)

Error getting the team names in game 24
Error getting the team names in game 48
Error getting the team names in game 93
Error getting the team names in game 356


In [ ]:
new_df.head()

,game_number,home_team_name,away_team_name,home_goals,away_goals,home_shots,away_shots,home_hits,away_hits,home_ppg,away_ppg
0,1,Toronto Maple Leafs,Montréal Canadiens,3,2,26,36,18.0,34.0,1.0,1.0
1,2,Washington Capitals,Boston Bruins,7,0,37,25,16.0,28.0,4.0,0.0
2,3,Vancouver Canucks,Calgary Flames,5,2,23,35,16.0,17.0,0.0,0.0
3,4,San Jose Sharks,Anaheim Ducks,2,5,33,15,21.0,25.0,0.0,2.0
4,5,Buffalo Sabres,Boston Bruins,0,4,32,26,25.0,13.0,0.0,1.0


In [ ]:
df_2018[df_2018['game_number'] == 47]

,player_id,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffWins,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,faceOffPct,pim,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,evenStrengthSavePercentage,away_team_name,away_team_id,season,game_number,home_team_name,home_team_id,shortHandedSavePercentage
1789,ID8471702,19:46,0,0,3,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,15:46,0:00,4:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Washington Capitals,15.0,2018,47,NaN,NaN,NaN
1790,ID8476905,9:18,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8:23,0:00,0:55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Washington Capitals,15.0,2018,47,NaN,NaN,NaN
1791,ID8468498,14:05,0,0,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,11:19,0:00,2:46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Washington Capitals,15.0,2018,47,NaN,NaN,NaN
1792,ID8475758,9:34,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,6:21,0:00,3:13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Washington Capitals,15.0,2018,47,NaN,NaN,NaN
1793,ID8476436,9:06,0,0,1,3.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,-2.0,9:06,0:00,0:00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Washington Capitals,15.0,2018,47,NaN,NaN,NaN
1794,ID8471214,23:57,0,0,2,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,17:45,6:12,0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Washington Capitals,15.0,2018,47,NaN,NaN,NaN
1795,ID8471698,18:46,0,0,0,1.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,1.0,-1.0,13:58,3:49,0:59,50.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Washington Capitals,15.0,2018,47,NaN,NaN,NaN
1796,ID8475744,20:09,0,0,1,3.0,0.0,0.0,2.0,3.0,17.0,3.0,0.0,0.0,0.0,0.0,-1.0,16:23,3:13,0:33,17.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Washington Capitals,15.0,2018,47,NaN,NaN,NaN
1797,ID8477944,16:12,0,0,2,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,14:22,1:50,0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Washington Capitals,15.0,2018,47,NaN,NaN,NaN
1798,ID8477444,10:55,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-2.0,9:52,0:51,0:12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Washington Capitals,15.0,2018,47,NaN,NaN,NaN


In [ ]:
by_game_away = pd.DataFrame(df_2018.groupby(['game_number'])).groupby(['away_team_name']).sum()
by_game_home = pd.DataFrame(df_2018.groupby(['game_number'])).groupby(['home_team_name']).sum()
by_game = pd.concat([by_game_away, by_game_home])
by_game.head(10)
#Shots are wrong...it's the whole game; divide by 2
#game_number is summed
#use sum of goalie time on ice to figure out if it went to over time; need this for good predictions
#faceoff % is summed
#pim is probably wrong


KeyError: ignored

In [ ]:
df_2018.sample(10)

,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffWins,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,faceOffPct,pim,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,evenStrengthSavePercentage,away_team_name,away_team_id,season,game_number,home_team_name,home_team_id,shortHandedSavePercentage
player_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ID8478472,11:14,0,1,5,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,11:03,0:11,0:00,100.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,0.0325,Tampa Bay Lightning,14.0,NaN
ID8477935,12:20,0,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,11:00,1:20,0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,0.1008,Calgary Flames,20.0,NaN
ID8476881,21:49,0,0,0,0.0,0.0,0.0,0.0,14.0,22.0,2.0,1.0,0.0,0.0,1.0,-1.0,14:53,4:19,2:37,63.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,0.0902,San Jose Sharks,28.0,NaN
ID8474066,14:48,0,0,0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-1.0,14:18,0:00,0:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,0.0969,New York Islanders,2.0,NaN
ID8476925,17:36,0,0,3,1.0,0.0,0.0,0.0,6.0,12.0,0.0,0.0,0.0,0.0,1.0,0.0,10:26,4:59,2:11,50.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,0.0202,Nashville Predators,18.0,NaN
ID8476981,17:55,0,1,4,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,16:27,0:50,0:38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Columbus Blue Jackets,29.0,2018,0.0900,NaN,NaN,NaN
ID8476882,19:44,1,2,2,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,15:55,2:24,1:25,50.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Carolina Hurricanes,12.0,2018,0.0823,NaN,NaN,NaN
ID8476443,15:51,0,0,2,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,-2.0,14:56,0:00,0:55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,0.0051,Montréal Canadiens,8.0,NaN
ID8477401,5:55,0,0,1,1.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5:55,0:00,0:00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chicago Blackhawks,16.0,2018,0.1266,NaN,NaN,NaN
